# Title
[]()

In [1]:

import sys
sys.path.append('../src')
sys.path.append(r"/home/silvhua/custom_python")
from silvhua import *

In [2]:
# set the option to wrap text within cells
pd.set_option('display.max_colwidth', 100)
# pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_rows', 20)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)

# Initialize

In [3]:

result_dict = dict()

# Load JSON

In [4]:
filename = '/home/silvhua/repositories/pubmed-search/data/pubmed_results_2024-04-06_235718.json'
pubmed_records = load_json(filename, filepath='')
pubmed_records

[{'article_title': "Exploring home rehabilitation therapists' experiences of supporting older persons to physical exercise after acute hospitalization: a qualitative interview study.",
  'abstract': "PURPOSE: After hospitalization, older persons may face a decline in physical function and daily independence. In-hospital exercise interventions can mitigate this decline, and continued support from primary healthcare post-discharge may enhance sustainability. This study aimed to explore home rehabilitation therapists' experiences of supporting physical exercise after acute hospitalization, including exercise programs initiated during hospital stay. METHODS: This qualitative study was conducted alongside a randomized-controlled trial to investigate prerequisites for a transitional care intervention. Twelve interviews were conducted with physiotherapists, occupational therapists, and managers across seven rehabilitation therapy services in Stockholm, Sweden. Data were analyzed using reflexi

# Haystack
https://haystack.deepset.ai/tutorials/39_embedding_metadata_for_improved_retrieval
https://colab.research.google.com/github/deepset-ai/haystack-tutorials/blob/main/tutorials/39_Embedding_Metadata_for_Improved_Retrieval.ipynb#scrollTo=J6GCkMgP6j6K

In [5]:
from haystack import Pipeline
from haystack.components.preprocessors import DocumentCleaner, DocumentSplitter
from haystack.components.embedders import SentenceTransformersDocumentEmbedder
from haystack.components.writers import DocumentWriter
from haystack.document_stores.types import DuplicatePolicy
from haystack.utils import ComponentDevice

from haystack import Document
from haystack.document_stores.in_memory import InMemoryDocumentStore


def create_indexing_pipeline(document_store, metadata_fields_to_embed=None):
    document_cleaner = DocumentCleaner()
    document_splitter = DocumentSplitter(split_by="sentence", split_length=2)
    document_embedder = SentenceTransformersDocumentEmbedder(
        # model="thenlper/gte-large", 
        meta_fields_to_embed=metadata_fields_to_embed
    )
    document_writer = DocumentWriter(document_store=document_store, policy=DuplicatePolicy.OVERWRITE)

    indexing_pipeline = Pipeline()
    indexing_pipeline.add_component("cleaner", document_cleaner)
    indexing_pipeline.add_component("splitter", document_splitter)
    indexing_pipeline.add_component("embedder", document_embedder)
    indexing_pipeline.add_component("writer", document_writer)

    indexing_pipeline.connect("cleaner", "splitter")
    indexing_pipeline.connect("splitter", "embedder")
    indexing_pipeline.connect("embedder", "writer")

    return indexing_pipeline

document_store_with_embedded_metadata = InMemoryDocumentStore(embedding_similarity_function="cosine")
indexing_with_metadata_pipeline = create_indexing_pipeline(
    document_store=document_store_with_embedded_metadata, metadata_fields_to_embed=["article_title"]
)

/home/silvhua/.pubmed/lib/python3.10/site-packages/transformers/utils/generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(


In [6]:
indexing_with_metadata_pipeline 

In [7]:
def create_docs(
        dictionary_list, content_key='abstract', meta_keys=['article_title', 'keywords', 'mesh_headings', 'journal']
        ):
    raw_docs = []
    for dictionary in dictionary_list:
        doc = Document(content=dictionary[content_key], meta={key: dictionary[key] for key in meta_keys})
        raw_docs.append(doc)
    return raw_docs

raw_docs = create_docs(pubmed_records)
raw_docs

[Document(id=2239987aee4836fd54fe23937cd2667882b4c6ca487004817320c1c55f26578b, content: 'PURPOSE: After hospitalization, older persons may face a decline in physical function and daily inde...', meta: {'article_title': "Exploring home rehabilitation therapists' experiences of supporting older persons to physical exercise after acute hospitalization: a qualitative interview study.", 'keywords': ['Exercise', 'Older adults', 'Primary care', 'Qualitative study', 'Rehabilitation therapy', 'Transitional care'], 'mesh_headings': None, 'journal': 'European geriatric medicine'}),
 Document(id=bf9b430c22e923d651790a600dae2892a6cf321a409676a4e49c04d65e19c3da, content: 'BACKGROUND: There are few reports on transthoracic echocardiography (TTE) for the evaluation of valv...', meta: {'article_title': 'Current awareness and use of transthoracic echocardiography in evaluation of valvular heart disease in Kumamoto Prefecture -a report from the Kumamoto cardiovascular echocardiography standardization pro

In [8]:
print(raw_docs[0])

Document(id=2239987aee4836fd54fe23937cd2667882b4c6ca487004817320c1c55f26578b, content: 'PURPOSE: After hospitalization, older persons may face a decline in physical function and daily inde...', meta: {'article_title': "Exploring home rehabilitation therapists' experiences of supporting older persons to physical exercise after acute hospitalization: a qualitative interview study.", 'keywords': ['Exercise', 'Older adults', 'Primary care', 'Qualitative study', 'Rehabilitation therapy', 'Transitional care'], 'mesh_headings': None, 'journal': 'European geriatric medicine'})


In [9]:
raw_docs[0].content

"PURPOSE: After hospitalization, older persons may face a decline in physical function and daily independence. In-hospital exercise interventions can mitigate this decline, and continued support from primary healthcare post-discharge may enhance sustainability. This study aimed to explore home rehabilitation therapists' experiences of supporting physical exercise after acute hospitalization, including exercise programs initiated during hospital stay. METHODS: This qualitative study was conducted alongside a randomized-controlled trial to investigate prerequisites for a transitional care intervention. Twelve interviews were conducted with physiotherapists, occupational therapists, and managers across seven rehabilitation therapy services in Stockholm, Sweden. Data were analyzed using reflexive thematic analysis. RESULTS: The analysis generated the theme Striving for individualized support for physical exercise, although limited resources and a fragmented home care risk to direct support

In [10]:
indexing_with_metadata_pipeline.run({"cleaner": {"documents": raw_docs}})

/home/silvhua/.pubmed/lib/python3.10/site-packages/transformers/utils/generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
/home/silvhua/.pubmed/lib/python3.10/site-packages/transformers/utils/generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(


Batches:   0%|          | 0/4 [00:00<?, ?it/s]

{'writer': {'documents_written': 125}}

In [13]:
indexing_with_metadata_pipeline.to_dict()

{'metadata': {},
 'max_loops_allowed': 100,
 'components': {'cleaner': {'type': 'haystack.components.preprocessors.document_cleaner.DocumentCleaner',
   'init_parameters': {'remove_empty_lines': True,
    'remove_extra_whitespaces': True,
    'remove_repeated_substrings': False,
    'remove_substrings': None,
    'remove_regex': None}},
  'splitter': {'type': 'haystack.components.preprocessors.document_splitter.DocumentSplitter',
   'init_parameters': {'split_by': 'sentence',
    'split_length': 2,
    'split_overlap': 0}},
  'embedder': {'type': 'haystack.components.embedders.sentence_transformers_document_embedder.SentenceTransformersDocumentEmbedder',
   'init_parameters': {'model': 'sentence-transformers/all-mpnet-base-v2',
    'device': 'cpu',
    'token': None,
    'prefix': '',
    'suffix': '',
    'batch_size': 32,
    'progress_bar': True,
    'normalize_embeddings': False,
    'meta_fields_to_embed': ['article_title'],
    'embedding_separator': '\n'}},
  'writer': {'type': 

In [12]:
vars(indexing_with_metadata_pipeline)

{'_telemetry_runs': 1,
 '_last_telemetry_sent': datetime.datetime(2024, 4, 7, 17, 4, 56, 661567),
 'metadata': {},
 'max_loops_allowed': 100,
 'graph': <networkx.classes.multidigraph.MultiDiGraph at 0x7f99d23b3250>,
 '_connections': [cleaner.documents (List[Document]) --> (List[Document]) splitter.documents,
  splitter.documents (List[Document]) --> (List[Document]) embedder.documents,
  embedder.documents (List[Document]) --> (List[Document]) writer.documents],
 '_mandatory_connections': defaultdict(list,
             {'splitter': [cleaner.documents (List[Document]) --> (List[Document]) splitter.documents],
              'embedder': [splitter.documents (List[Document]) --> (List[Document]) embedder.documents],
              'writer': [embedder.documents (List[Document]) --> (List[Document]) writer.documents],
              'cleaner': []}),
 '_debug': {},
 '_debug_path': PosixPath('.haystack_debug')}

## Serialize and de-serialize

In [15]:
save_to_json(
    indexing_with_metadata_pipeline.to_dict(), filename='indexing_pipeline', append_version=True, path='../pipeline'
)

Object saved as JSON: ../pipeline/indexing_pipeline_2024-04-07_170902.json


In [17]:
loaded_indexing_json = load_json(
    filename='indexing_pipeline_2024-04-07_170902.json', filepath='../pipeline'
)

In [18]:
loaded_index_pipeline = Pipeline().from_dict(loaded_indexing_json)
loaded_index_pipeline 

## Retriever

In [21]:
from haystack.components.embedders import SentenceTransformersTextEmbedder
from haystack.components.retrievers.in_memory import InMemoryEmbeddingRetriever

retrieval_pipeline = Pipeline()
retrieval_pipeline.add_component("text_embedder", SentenceTransformersTextEmbedder(
    # model="thenlper/gte-large"
    ))
retrieval_pipeline.add_component(
    "retriever_with_embeddings",
    InMemoryEmbeddingRetriever(document_store=document_store_with_embedded_metadata, scale_score=False, top_k=3),
)

retrieval_pipeline.connect("text_embedder", "retriever_with_embeddings")

In [22]:
save_to_json(
    retrieval_pipeline.to_dict(), filename='retrieval_pipeline', append_version=True, path='../pipeline'
)

Object saved as JSON: ../pipeline/retrieval_pipeline_2024-04-07_171557.json


In [23]:
query = 'blood glucose control'
result = retrieval_pipeline.run({"text_embedder": {"text": query}})
result

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

{'retriever_with_embeddings': {'documents': [Document(id=a3db6d60931498e26902aaeb51d8bf9ab9953bb398fed1aae602e43f0721c317, content: ' METHODS: The sample comprised 89 persons (41 females; mean age 40.1 years at baseline) who particip...', meta: {'article_title': 'Longitudinal association between fitness and metabolic syndrome: a population-based study over 29 years follow-up.', 'keywords': ['Exercise', 'Fitness', 'Health', 'Longitudinal study', 'Metabolic syndrome'], 'mesh_headings': None, 'journal': 'BMC public health', 'source_id': 'c3e91df667b0d62f8ef350dac60e6579914e0bf1b922d6a6ff82f3d521b77b6e'}, score: 0.2697038913461883),
   Document(id=32fed1100a4a577f88d251753e33113582fd5899111c72edc88a4a7bf84eb646, content: ' Fitness (predictor variable) was assessed using 15 standardized and validated tests that measured s...', meta: {'article_title': 'Longitudinal association between fitness and metabolic syndrome: a population-based study over 29 years follow-up.', 'keywords': ['Exercise',

In [27]:
vars(document_store_with_embedded_metadata)

{'storage': {'194498ea5fdfbac89cf1684795d3fc9d4f302852137e2b2d8b7dfbec1a918f71': Document(id=194498ea5fdfbac89cf1684795d3fc9d4f302852137e2b2d8b7dfbec1a918f71, content: 'PURPOSE: After hospitalization, older persons may face a decline in physical function and daily inde...', meta: {'article_title': "Exploring home rehabilitation therapists' experiences of supporting older persons to physical exercise after acute hospitalization: a qualitative interview study.", 'keywords': ['Exercise', 'Older adults', 'Primary care', 'Qualitative study', 'Rehabilitation therapy', 'Transitional care'], 'mesh_headings': None, 'journal': 'European geriatric medicine', 'source_id': '2239987aee4836fd54fe23937cd2667882b4c6ca487004817320c1c55f26578b'}, embedding: vector of size 768),
  '4f9265af24ee7cd2d0aba3fd8718a3894c7c29a3b788386389da7eddfea9f2b1': Document(id=4f9265af24ee7cd2d0aba3fd8718a3894c7c29a3b788386389da7eddfea9f2b1, content: ' This study aimed to explore home rehabilitation therapists' experiences

In [28]:
document_store_with_embedded_metadata.storage

{'194498ea5fdfbac89cf1684795d3fc9d4f302852137e2b2d8b7dfbec1a918f71': Document(id=194498ea5fdfbac89cf1684795d3fc9d4f302852137e2b2d8b7dfbec1a918f71, content: 'PURPOSE: After hospitalization, older persons may face a decline in physical function and daily inde...', meta: {'article_title': "Exploring home rehabilitation therapists' experiences of supporting older persons to physical exercise after acute hospitalization: a qualitative interview study.", 'keywords': ['Exercise', 'Older adults', 'Primary care', 'Qualitative study', 'Rehabilitation therapy', 'Transitional care'], 'mesh_headings': None, 'journal': 'European geriatric medicine', 'source_id': '2239987aee4836fd54fe23937cd2667882b4c6ca487004817320c1c55f26578b'}, embedding: vector of size 768),
 '4f9265af24ee7cd2d0aba3fd8718a3894c7c29a3b788386389da7eddfea9f2b1': Document(id=4f9265af24ee7cd2d0aba3fd8718a3894c7c29a3b788386389da7eddfea9f2b1, content: ' This study aimed to explore home rehabilitation therapists' experiences of supportin

In [29]:
save_to_json(
    document_store_with_embedded_metadata.to_dict(), 'document_store', append_version=True, path='../pipeline'
)

Object saved as JSON: ../pipeline/document_store_2024-04-07_172039.json


# Try with larger data set

## Iteration 1: did not work without documents

In [3]:
loaded_indexing_json = load_json(
    filename='indexing_pipeline_2024-04-07_170902.json', filepath='../pipeline'
)

In [5]:
from haystack import Pipeline
from haystack.components.preprocessors import DocumentCleaner, DocumentSplitter
from haystack.components.embedders import SentenceTransformersDocumentEmbedder
from haystack.components.writers import DocumentWriter
from haystack.document_stores.types import DuplicatePolicy
from haystack.utils import ComponentDevice

from haystack import Document
from haystack.document_stores.in_memory import InMemoryDocumentStore

loaded_index_pipeline = Pipeline().from_dict(loaded_indexing_json)
loaded_index_pipeline 

/home/silvhua/.pubmed/lib/python3.10/site-packages/transformers/utils/generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(


In [6]:
from haystack.components.embedders import SentenceTransformersTextEmbedder
from haystack.components.retrievers.in_memory import InMemoryEmbeddingRetriever

retrieval_pipeline_json = load_json(
    filename='retrieval_pipeline_2024-04-07_171557.json', filepath='../pipeline'
)
loaded_retrieval_pipeline = Pipeline().from_dict(retrieval_pipeline_json)


In [8]:
query = 'blood glucose control'
result = loaded_retrieval_pipeline.run({"text_embedder": {"text": query}})
result

/home/silvhua/.pubmed/lib/python3.10/site-packages/transformers/utils/generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
/home/silvhua/.pubmed/lib/python3.10/site-packages/transformers/utils/generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

No Documents found with embeddings. Returning empty list. To generate embeddings, use a DocumentEmbedder.


{'retriever_with_embeddings': {'documents': []}}

## Iteration 2: Unable to load document store from InMemoryDocumentStore

In [4]:
loaded_indexing_json = load_json(
    filename='indexing_pipeline_2024-04-07_170902.json', filepath='../pipeline'
)

In [5]:
from haystack import Pipeline
from haystack.components.preprocessors import DocumentCleaner, DocumentSplitter
from haystack.components.embedders import SentenceTransformersDocumentEmbedder
from haystack.components.writers import DocumentWriter
from haystack.document_stores.types import DuplicatePolicy
from haystack.utils import ComponentDevice

from haystack import Document
from haystack.document_stores.in_memory import InMemoryDocumentStore

loaded_index_pipeline = Pipeline().from_dict(loaded_indexing_json)
loaded_index_pipeline 

In [7]:
def create_docs(
        dictionary_list, content_key='abstract', meta_keys=['article_title', 'keywords', 'mesh_headings', 'journal']
        ):
    raw_docs = []
    for dictionary in dictionary_list:
        doc = Document(content=dictionary[content_key], meta={key: dictionary[key] for key in meta_keys})
        raw_docs.append(doc)
    return raw_docs

filename = '/home/silvhua/repositories/pubmed-search/data/pubmed_results_2024-04-06_235718.json'
pubmed_records = load_json(filename, filepath='')
raw_docs = create_docs(pubmed_records)
raw_docs

[Document(id=2239987aee4836fd54fe23937cd2667882b4c6ca487004817320c1c55f26578b, content: 'PURPOSE: After hospitalization, older persons may face a decline in physical function and daily inde...', meta: {'article_title': "Exploring home rehabilitation therapists' experiences of supporting older persons to physical exercise after acute hospitalization: a qualitative interview study.", 'keywords': ['Exercise', 'Older adults', 'Primary care', 'Qualitative study', 'Rehabilitation therapy', 'Transitional care'], 'mesh_headings': None, 'journal': 'European geriatric medicine'}),
 Document(id=bf9b430c22e923d651790a600dae2892a6cf321a409676a4e49c04d65e19c3da, content: 'BACKGROUND: There are few reports on transthoracic echocardiography (TTE) for the evaluation of valv...', meta: {'article_title': 'Current awareness and use of transthoracic echocardiography in evaluation of valvular heart disease in Kumamoto Prefecture -a report from the Kumamoto cardiovascular echocardiography standardization pro

In [9]:
loaded_index_pipeline.run({"cleaner": {"documents": raw_docs}})

/home/silvhua/.pubmed/lib/python3.10/site-packages/transformers/utils/generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
/home/silvhua/.pubmed/lib/python3.10/site-packages/transformers/utils/generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(


Batches:   0%|          | 0/4 [00:00<?, ?it/s]

{'writer': {'documents_written': 125}}

In [10]:
vars(loaded_index_pipeline)

{'_telemetry_runs': 1,
 '_last_telemetry_sent': datetime.datetime(2024, 4, 7, 19, 30, 58, 119228),
 'metadata': {},
 'max_loops_allowed': 100,
 'graph': <networkx.classes.multidigraph.MultiDiGraph at 0x7fbc4c838ee0>,
 '_connections': [cleaner.documents (List[Document]) --> (List[Document]) splitter.documents,
  splitter.documents (List[Document]) --> (List[Document]) embedder.documents,
  embedder.documents (List[Document]) --> (List[Document]) writer.documents],
 '_mandatory_connections': defaultdict(list,
             {'splitter': [cleaner.documents (List[Document]) --> (List[Document]) splitter.documents],
              'embedder': [splitter.documents (List[Document]) --> (List[Document]) embedder.documents],
              'writer': [embedder.documents (List[Document]) --> (List[Document]) writer.documents],
              'cleaner': []}),
 '_debug': {},
 '_debug_path': PosixPath('.canals_debug')}

In [16]:
print(type(loaded_index_pipeline._mandatory_connections['writer'][0]))
loaded_index_pipeline._mandatory_connections['writer'][0]

<class 'haystack.core.component.connection.Connection'>


embedder.documents (List[Document]) --> (List[Document]) writer.documents

In [17]:
from haystack.components.embedders import SentenceTransformersTextEmbedder
from haystack.components.retrievers.in_memory import InMemoryEmbeddingRetriever

retrieval_pipeline_json = load_json(
    filename='retrieval_pipeline_2024-04-07_171557.json', filepath='../pipeline'
)
loaded_retrieval_pipeline = Pipeline().from_dict(retrieval_pipeline_json)


In [14]:
vars(loaded_retrieval_pipeline)

{'_telemetry_runs': 1,
 '_last_telemetry_sent': datetime.datetime(2024, 4, 7, 17, 29, 56, 984501),
 'metadata': {},
 'max_loops_allowed': 100,
 'graph': <networkx.classes.multidigraph.MultiDiGraph at 0x7fd7807ee9e0>,
 '_connections': [text_embedder.embedding (List[float]) --> (List[float]) retriever_with_embeddings.query_embedding],
 '_mandatory_connections': defaultdict(list,
             {'retriever_with_embeddings': [text_embedder.embedding (List[float]) --> (List[float]) retriever_with_embeddings.query_embedding],
              'text_embedder': []}),
 '_debug': {},
 '_debug_path': PosixPath('.canals_debug')}

In [20]:
document_store_json = load_json(
    'document_store_2024-04-07_172039.json', filepath='../pipeline'
)
document_store = InMemoryDocumentStore.from_dict(document_store_json)
vars(document_store)

{'storage': {},
 '_bm25_tokenization_regex': '(?u)\\b\\w\\w+\\b',
 'tokenizer': <function Pattern.findall(string, pos=0, endpos=9223372036854775807)>,
 'bm25_algorithm': haystack_bm25.rank_bm25.BM25Okapi,
 'bm25_parameters': {},
 'embedding_similarity_function': 'cosine'}

In [21]:
vars(loaded_index_pipeline)

{'_telemetry_runs': 1,
 '_last_telemetry_sent': datetime.datetime(2024, 4, 7, 19, 30, 58, 119228),
 'metadata': {},
 'max_loops_allowed': 100,
 'graph': <networkx.classes.multidigraph.MultiDiGraph at 0x7fbc4c838ee0>,
 '_connections': [cleaner.documents (List[Document]) --> (List[Document]) splitter.documents,
  splitter.documents (List[Document]) --> (List[Document]) embedder.documents,
  embedder.documents (List[Document]) --> (List[Document]) writer.documents],
 '_mandatory_connections': defaultdict(list,
             {'splitter': [cleaner.documents (List[Document]) --> (List[Document]) splitter.documents],
              'embedder': [splitter.documents (List[Document]) --> (List[Document]) embedder.documents],
              'writer': [embedder.documents (List[Document]) --> (List[Document]) writer.documents],
              'cleaner': []}),
 '_debug': {},
 '_debug_path': PosixPath('.canals_debug')}

In [24]:
vars(loaded_index_pipeline._connections[-1])

{'sender': 'embedder',
 'sender_socket': OutputSocket(name='documents', type=typing.List[haystack.dataclasses.document.Document], receivers=['writer']),
 'receiver': 'writer',
 'receiver_socket': InputSocket(name='documents', type=typing.List[haystack.dataclasses.document.Document], default_value=<class 'haystack.core.component.sockets._empty'>, is_variadic=False, senders=['embedder'])}

In [18]:
query = 'blood glucose control'
result = loaded_retrieval_pipeline.run({"text_embedder": {"text": query}})
result

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

No Documents found with embeddings. Returning empty list. To generate embeddings, use a DocumentEmbedder.


{'retriever_with_embeddings': {'documents': []}}

## iteration 3

In [22]:
document_store_json = load_json(
    'document_store_2024-04-07_172039.json', filepath='../pipeline'
)
document_store = InMemoryDocumentStore.from_dict(document_store_json)
vars(document_store)

{'storage': {},
 '_bm25_tokenization_regex': '(?u)\\b\\w\\w+\\b',
 'tokenizer': <function Pattern.findall(string, pos=0, endpos=9223372036854775807)>,
 'bm25_algorithm': haystack_bm25.rank_bm25.BM25Okapi,
 'bm25_parameters': {},
 'embedding_similarity_function': 'cosine'}

In [31]:
from haystack import Pipeline
from haystack.components.preprocessors import DocumentCleaner, DocumentSplitter
from haystack.components.embedders import SentenceTransformersDocumentEmbedder
from haystack.components.writers import DocumentWriter
from haystack.document_stores.types import DuplicatePolicy
from haystack.utils import ComponentDevice

from haystack import Document
from haystack.document_stores.in_memory import InMemoryDocumentStore

document_store_with_embedded_metadata = InMemoryDocumentStore(embedding_similarity_function="cosine")
document_writer = DocumentWriter(document_store=document_store, policy=DuplicatePolicy.OVERWRITE)

loaded_indexing_json = load_json(
    filename='indexing_pipeline_2024-04-07_170902.json', filepath='../pipeline'
)
loaded_index_pipeline = Pipeline().from_dict(loaded_indexing_json, kwargs={'writer': document_writer})
loaded_index_pipeline 

In [32]:
loaded_index_pipeline.run({"cleaner": {"documents": raw_docs}})

Batches:   0%|          | 0/4 [00:00<?, ?it/s]

{'writer': {'documents_written': 125}}

In [39]:
vars(document_store_with_embedded_metadata)

{'storage': {},
 '_bm25_tokenization_regex': '(?u)\\b\\w\\w+\\b',
 'tokenizer': <function Pattern.findall(string, pos=0, endpos=9223372036854775807)>,
 'bm25_algorithm': haystack_bm25.rank_bm25.BM25Okapi,
 'bm25_parameters': {},
 'embedding_similarity_function': 'cosine'}

In [40]:
from haystack.components.embedders import SentenceTransformersTextEmbedder
from haystack.components.retrievers.in_memory import InMemoryEmbeddingRetriever

retrieval_pipeline_json = load_json(
    filename='retrieval_pipeline_2024-04-07_171557.json', filepath='../pipeline'
)
retriever = InMemoryEmbeddingRetriever(document_store=document_store_with_embedded_metadata, scale_score=False, top_k=3)
loaded_retrieval_pipeline = Pipeline().from_dict(
    retrieval_pipeline_json, 
    kwargs={'retriever_with_embeddings': retriever}
    )
vars(loaded_retrieval_pipeline)

{'_telemetry_runs': 0,
 '_last_telemetry_sent': None,
 'metadata': {},
 'max_loops_allowed': 100,
 'graph': <networkx.classes.multidigraph.MultiDiGraph at 0x7fbc3c34bd00>,
 '_connections': [text_embedder.embedding (List[float]) --> (List[float]) retriever_with_embeddings.query_embedding],
 '_mandatory_connections': defaultdict(list,
             {'retriever_with_embeddings': [text_embedder.embedding (List[float]) --> (List[float]) retriever_with_embeddings.query_embedding]}),
 '_debug': {},
 '_debug_path': PosixPath('.canals_debug')}

In [41]:
vars(document_store_with_embedded_metadata)

{'storage': {},
 '_bm25_tokenization_regex': '(?u)\\b\\w\\w+\\b',
 'tokenizer': <function Pattern.findall(string, pos=0, endpos=9223372036854775807)>,
 'bm25_algorithm': haystack_bm25.rank_bm25.BM25Okapi,
 'bm25_parameters': {},
 'embedding_similarity_function': 'cosine'}

In [42]:
query = 'blood glucose control'
result = loaded_retrieval_pipeline.run({"text_embedder": {"text": query}})
result

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

No Documents found with embeddings. Returning empty list. To generate embeddings, use a DocumentEmbedder.


{'retriever_with_embeddings': {'documents': []}}

## Iteration 4

In [48]:
from haystack import Pipeline
from haystack.components.preprocessors import DocumentCleaner, DocumentSplitter
from haystack.components.embedders import SentenceTransformersDocumentEmbedder
from haystack.components.writers import DocumentWriter
from haystack.document_stores.types import DuplicatePolicy
from haystack.utils import ComponentDevice

from haystack import Document
from haystack.document_stores.in_memory import InMemoryDocumentStore


def create_indexing_pipeline(document_store, metadata_fields_to_embed=None):
    document_cleaner = DocumentCleaner()
    document_splitter = DocumentSplitter(split_by="sentence", split_length=2)
    document_embedder = SentenceTransformersDocumentEmbedder(
        # model="thenlper/gte-large", 
        meta_fields_to_embed=metadata_fields_to_embed
    )
    document_writer = DocumentWriter(document_store=document_store, policy=DuplicatePolicy.OVERWRITE)

    indexing_pipeline = Pipeline()
    indexing_pipeline.add_component("cleaner", document_cleaner)
    indexing_pipeline.add_component("splitter", document_splitter)
    indexing_pipeline.add_component("embedder", document_embedder)
    indexing_pipeline.add_component("writer", document_writer)

    indexing_pipeline.connect("cleaner", "splitter")
    indexing_pipeline.connect("splitter", "embedder")
    indexing_pipeline.connect("embedder", "writer")

    return indexing_pipeline

def create_docs(
        dictionary_list, content_key='abstract', meta_keys=['article_title', 'keywords', 'mesh_headings', 'journal']
        ):
    raw_docs = []
    for dictionary in dictionary_list:
        doc = Document(content=dictionary[content_key], meta={key: dictionary[key] for key in meta_keys})
        raw_docs.append(doc)
    return raw_docs

filename = '/home/silvhua/repositories/pubmed-search/data/pubmed_results_2024-04-06_235718.json'
pubmed_records = load_json(filename, filepath='')

document_store_with_embedded_metadata = InMemoryDocumentStore(embedding_similarity_function="cosine")
indexing_with_metadata_pipeline = create_indexing_pipeline(
    document_store=document_store_with_embedded_metadata, metadata_fields_to_embed=["article_title"]
)
raw_docs = create_docs(pubmed_records)
document_store_with_embedded_metadata.storage

{}

In [49]:

indexing_with_metadata_pipeline.run({"cleaner": {"documents": raw_docs}})

Batches:   0%|          | 0/4 [00:00<?, ?it/s]

{'writer': {'documents_written': 125}}

In [50]:
document_store_with_embedded_metadata.storage

{'194498ea5fdfbac89cf1684795d3fc9d4f302852137e2b2d8b7dfbec1a918f71': Document(id=194498ea5fdfbac89cf1684795d3fc9d4f302852137e2b2d8b7dfbec1a918f71, content: 'PURPOSE: After hospitalization, older persons may face a decline in physical function and daily inde...', meta: {'article_title': "Exploring home rehabilitation therapists' experiences of supporting older persons to physical exercise after acute hospitalization: a qualitative interview study.", 'keywords': ['Exercise', 'Older adults', 'Primary care', 'Qualitative study', 'Rehabilitation therapy', 'Transitional care'], 'mesh_headings': None, 'journal': 'European geriatric medicine', 'source_id': '2239987aee4836fd54fe23937cd2667882b4c6ca487004817320c1c55f26578b'}, embedding: vector of size 768),
 '4f9265af24ee7cd2d0aba3fd8718a3894c7c29a3b788386389da7eddfea9f2b1': Document(id=4f9265af24ee7cd2d0aba3fd8718a3894c7c29a3b788386389da7eddfea9f2b1, content: ' This study aimed to explore home rehabilitation therapists' experiences of supportin

In [51]:
from haystack.components.embedders import SentenceTransformersTextEmbedder
from haystack.components.retrievers.in_memory import InMemoryEmbeddingRetriever

retrieval_pipeline = Pipeline()
retrieval_pipeline.add_component("text_embedder", SentenceTransformersTextEmbedder(
    # model="thenlper/gte-large"
    ))
retrieval_pipeline.add_component(
    "retriever_with_embeddings",
    InMemoryEmbeddingRetriever(document_store=document_store_with_embedded_metadata, scale_score=False, top_k=3),
)

retrieval_pipeline.connect("text_embedder", "retriever_with_embeddings")

In [52]:
query = 'athlete upper limb'
result = retrieval_pipeline.run({"text_embedder": {"text": query}})
result

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

{'retriever_with_embeddings': {'documents': [Document(id=abac55c70138a1ed441c94160dabd7d947e9d3a0d29690d5cf9abc24c30dac98, content: ' Clinical history was obtained from all subjects, followed by dominant shoulder physical and ultraso...', meta: {'article_title': 'Clinical and ultrasonographic evaluation of uninjured dominant shoulder in amateur rugby players vs a control group: a pilot study.', 'keywords': ['Clinical exam of the shoulder', 'Rotator cuff injuries', 'Rugby', 'Shoulder injuries', 'Tendon calcification', 'Ultrasonography'], 'mesh_headings': None, 'journal': 'Journal of ultrasound', 'source_id': '3574c5cbf1923c2c083195be188a5332bc72445f9fcc77d2ec6000803bbddbff'}, score: 0.5044884706399849),
   Document(id=d26e5c1e1c16958f5da938d80e489fb5f10d5d7be8d76936ad084a7dddf3c99f, content: '05). CONCLUSIONS: Uninjured dominant shoulder of rugby players shows higher prevalence of clinical a...', meta: {'article_title': 'Clinical and ultrasonographic evaluation of uninjured dominant sho

## Iteration 5

In [2]:
from haystack import Pipeline
from haystack.components.preprocessors import DocumentCleaner, DocumentSplitter
from haystack.components.embedders import SentenceTransformersDocumentEmbedder
from haystack.components.writers import DocumentWriter
from haystack.document_stores.types import DuplicatePolicy
from haystack.utils import ComponentDevice

from haystack import Document
# from haystack.document_stores.in_memory import InMemoryDocumentStore
from haystack_integrations.document_stores.chroma import ChromaDocumentStore

def create_indexing_pipeline(document_store, metadata_fields_to_embed=None):
    document_cleaner = DocumentCleaner()
    document_splitter = DocumentSplitter(split_by="sentence", split_length=2)
    document_embedder = SentenceTransformersDocumentEmbedder(
        # model="thenlper/gte-large", 
        meta_fields_to_embed=metadata_fields_to_embed
    )
    document_writer = DocumentWriter(document_store=document_store, policy=DuplicatePolicy.OVERWRITE)

    indexing_pipeline = Pipeline()
    indexing_pipeline.add_component("cleaner", document_cleaner)
    indexing_pipeline.add_component("splitter", document_splitter)
    indexing_pipeline.add_component("embedder", document_embedder)
    indexing_pipeline.add_component("writer", document_writer)

    indexing_pipeline.connect("cleaner", "splitter")
    indexing_pipeline.connect("splitter", "embedder")
    indexing_pipeline.connect("embedder", "writer")

    return indexing_pipeline

def create_docs(
        dictionary_list, content_key='abstract', meta_keys=['article_title', 'keywords', 'mesh_headings', 'journal']
        ):
    raw_docs = []
    for dictionary in dictionary_list:
        doc = Document(content=dictionary[content_key], meta={key: dictionary[key] for key in meta_keys})
        raw_docs.append(doc)
    return raw_docs

filename = '/home/silvhua/repositories/pubmed-search/data/pubmed_results_2024-04-06_235718.json'
pubmed_records = load_json(filename, filepath='')

document_store = ChromaDocumentStore( # https://docs.haystack.deepset.ai/reference/integrations-chroma#chromadocumentstore
    collection_name='test_set', 
    persist_path='../data/processed/'
    )
indexing_with_metadata_pipeline = create_indexing_pipeline(
    document_store=document_store, metadata_fields_to_embed=["article_title"]
)
raw_docs = create_docs(pubmed_records)
document_store.storage

/home/silvhua/.pubmed/lib/python3.10/site-packages/transformers/utils/generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(


AttributeError: 'ChromaDocumentStore' object has no attribute 'storage'

In [4]:
vars(document_store)

{'_collection_name': 'test_set',
 '_embedding_function': 'default',
 '_embedding_function_params': {},
 '_persist_path': '../data/processed/',
 '_chroma_client': <chromadb.api.client.Client at 0x7f63ffc3b7c0>,
 '_collection': Collection(name=test_set)}

In [5]:

indexing_with_metadata_pipeline.run({"cleaner": {"documents": raw_docs}})

/home/silvhua/.pubmed/lib/python3.10/site-packages/transformers/utils/generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
/home/silvhua/.pubmed/lib/python3.10/site-packages/transformers/utils/generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(


Batches:   0%|          | 0/4 [00:00<?, ?it/s]

PipelineRuntimeError: writer raised 'ValueError: Expected metadata value to be a str, int, float or bool, got ['Exercise', 'Older adults', 'Primary care', 'Qualitative study', 'Rehabilitation therapy', 'Transitional care'] which is a <class 'list'>' 
Inputs: {'documents': [Document(id=194498ea5fdfbac89cf1684795d3fc9d4f302852137e2b2d8b7dfbec1a918f71, content: 'PURPOSE: After hospitalization, older persons may face a decline in physical function and daily inde...', meta: {'article_title': "Exploring home rehabilitation therapists' experiences of supporting older persons to physical exercise after acute hospitalization: a qualitative interview study.", 'keywords': ['Exercise', 'Older adults', 'Primary care', 'Qualitative study', 'Rehabilitation therapy', 'Transitional care'], 'mesh_headings': None, 'journal': 'European geriatric medicine', 'source_id': '2239987aee4836fd54fe23937cd2667882b4c6ca487004817320c1c55f26578b'}, embedding: vector of size 768), Document(id=4f9265af24ee7cd2d0aba3fd8

## Iteration 6 used Chroma Document Store

In [6]:
from haystack import Pipeline
from haystack.components.preprocessors import DocumentCleaner, DocumentSplitter
from haystack.components.embedders import SentenceTransformersDocumentEmbedder
from haystack.components.writers import DocumentWriter
from haystack.document_stores.types import DuplicatePolicy
from haystack.utils import ComponentDevice

from haystack import Document
# from haystack.document_stores.in_memory import InMemoryDocumentStore
from haystack_integrations.document_stores.chroma import ChromaDocumentStore

def create_indexing_pipeline(document_store, metadata_fields_to_embed=None):
    document_cleaner = DocumentCleaner()
    document_splitter = DocumentSplitter(split_by="sentence", split_length=2)
    document_embedder = SentenceTransformersDocumentEmbedder(
        # model="thenlper/gte-large", 
        meta_fields_to_embed=metadata_fields_to_embed
    )
    document_writer = DocumentWriter(document_store=document_store, policy=DuplicatePolicy.OVERWRITE)

    indexing_pipeline = Pipeline()
    indexing_pipeline.add_component("cleaner", document_cleaner)
    indexing_pipeline.add_component("splitter", document_splitter)
    indexing_pipeline.add_component("embedder", document_embedder)
    indexing_pipeline.add_component("writer", document_writer)

    indexing_pipeline.connect("cleaner", "splitter")
    indexing_pipeline.connect("splitter", "embedder")
    indexing_pipeline.connect("embedder", "writer")

    return indexing_pipeline

def create_docs(
        dictionary_list, content_key='abstract', meta_keys=[
            'article_title', 'journal', 
            # 'keywords', 'mesh_headings'
            ]
        ):
    raw_docs = []
    for dictionary in dictionary_list:
        doc = Document(content=dictionary[content_key], meta={key: dictionary[key] for key in meta_keys})
        raw_docs.append(doc)
    return raw_docs

filename = '/home/silvhua/repositories/pubmed-search/data/pubmed_results_2024-04-06_235718.json'
pubmed_records = load_json(filename, filepath='')

document_store = ChromaDocumentStore( # https://docs.haystack.deepset.ai/reference/integrations-chroma#chromadocumentstore
    collection_name='test_set', 
    persist_path='../data/processed/'
    )
indexing_with_metadata_pipeline = create_indexing_pipeline(
    document_store=document_store, metadata_fields_to_embed=["article_title"]
)
raw_docs = create_docs(pubmed_records)
vars(document_store)

{'_collection_name': 'test_set',
 '_embedding_function': 'default',
 '_embedding_function_params': {},
 '_persist_path': '../data/processed/',
 '_chroma_client': <chromadb.api.client.Client at 0x7f64c601b6a0>,
 '_collection': Collection(name=test_set)}

In [7]:

indexing_with_metadata_pipeline.run({"cleaner": {"documents": raw_docs}})

Batches:   0%|          | 0/4 [00:00<?, ?it/s]

{'writer': {'documents_written': 125}}

In [8]:
vars(document_store)

{'_collection_name': 'test_set',
 '_embedding_function': 'default',
 '_embedding_function_params': {},
 '_persist_path': '../data/processed/',
 '_chroma_client': <chromadb.api.client.Client at 0x7f64c601b6a0>,
 '_collection': Collection(name=test_set)}

In [11]:
from haystack_integrations.components.retrievers.chroma import ChromaEmbeddingRetriever

# https://docs.haystack.deepset.ai/docs/chromaembeddingretriever
retriever = ChromaEmbeddingRetriever(document_store=document_store, top_k=3)
retriever

In [12]:
vars(retriever)

{'filters': None,
 'top_k': 3,
 'document_store': <haystack_integrations.document_stores.chroma.document_store.ChromaDocumentStore at 0x7f64c601abf0>,
 '__canals_output__': {'documents': OutputSocket(name='documents', type=typing.List[haystack.dataclasses.document.Document], receivers=[])},
 '__canals_input__': {'query_embedding': InputSocket(name='query_embedding', type=typing.List[float], default_value=<class 'haystack.core.component.sockets._empty'>, is_variadic=False, senders=[]),
  'filters': InputSocket(name='filters', type=typing.Optional[typing.Dict[str, typing.Any]], default_value=None, is_variadic=False, senders=[]),
  'top_k': InputSocket(name='top_k', type=typing.Optional[int], default_value=None, is_variadic=False, senders=[])}}

In [13]:
from haystack.components.embedders import SentenceTransformersTextEmbedder
# from haystack.components.retrievers.in_memory import InMemoryEmbeddingRetriever

retrieval_pipeline = Pipeline()
retrieval_pipeline.add_component("text_embedder", SentenceTransformersTextEmbedder(
    # model="thenlper/gte-large"
    ))
retrieval_pipeline.add_component("retriever_with_embeddings", retriever)

retrieval_pipeline.connect("text_embedder", "retriever_with_embeddings")

In [14]:
query = 'athlete upper limb'
result = retrieval_pipeline.run({"text_embedder": {"text": query}})
result

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

{'retriever_with_embeddings': {'documents': [Document(id=ed6f8e0048f9ded9122d41519121008767ccf21cb84fa182f3a4835efc1578dd, content: ' Clinical history was obtained from all subjects, followed by dominant shoulder physical and ultraso...', meta: {'article_title': 'Clinical and ultrasonographic evaluation of uninjured dominant shoulder in amateur rugby players vs a control group: a pilot study.', 'journal': 'Journal of ultrasound', 'source_id': '7ad34a7145c28086e5c402f139796e40c033b3954dd92145792ac2b46ba9d7b2'}, score: 0.9910231232643127, embedding: vector of size 768),
   Document(id=b88aa01f498ae66c87b04da22e34ed64b89e07b3f9e3d1930724a77d9d8cfddf, content: '05). CONCLUSIONS: Uninjured dominant shoulder of rugby players shows higher prevalence of clinical a...', meta: {'article_title': 'Clinical and ultrasonographic evaluation of uninjured dominant shoulder in amateur rugby players vs a control group: a pilot study.', 'journal': 'Journal of ultrasound', 'source_id': '7ad34a7145c28086e5c

# *End of Page*